In [1]:
!pip install eli5

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

from xgboost import XGBRegressor

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dw_matrix/data/w_shoe_prices.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]

In [6]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-51.356432361171095, 0.7207640114801125)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features =  literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [8]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

342

In [9]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook( keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0]/df.shape[0] * 100

In [0]:
for key in keys_stat:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [12]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']
run_model(feats, model)

(-50.21165462032037, 1.1613611793913867)

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x]

In [0]:
feats_cat.remove('feat_catalog')

In [15]:
feats = feats_cat
model = XGBRegressor(max_depth=10, n_estimators=100, random_state=2020)
run_model(feats, model)

[16:47:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-43.933548143125776, 2.2572855392033384)

In [16]:
X = df [ feats ].values
y = df['prices_amountmin'].values
m = model.fit(X, y)

perm = PermutationImportance(m, random_state=0).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

[16:50:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.8483 ± 0.0288,brand_cat
0.4489 ± 0.0232,feat_brand_cat
0.0865 ± 0.0123,feat_heel height_cat
0.0717 ± 0.0053,feat_shoe size_cat
0.0486 ± 0.0026,feat_material_cat
0.0393 ± 0.0070,feat_gender_cat
0.0368 ± 0.0076,feat_condition_cat
0.0259 ± 0.0026,feat_color_cat
0.0222 ± 0.0025,feat_style_cat
0.0165 ± 0.0041,feat_trends_cat


In [17]:
feats = [
         'brand_cat',
]
model = XGBRegressor(max_depth=10, n_estimators=100, random_state=2020)
run_model(feats, model)

[16:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-43.92681324384391, 2.17219578598859)

In [19]:
feats = [
      'brand_cat',
      'feat_brand_cat',
      'feat_heel height_cat',
      'feat_shoe size_cat',
      'feat_material_cat',
      'feat_gender_cat'
]
model = XGBRegressor(max_depth=10, n_estimators=100, random_state=2020)
run_model(feats, model)

[17:00:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-43.45034355498619, 1.6000429194599366)